# Azure AI Search ile NVIDIA NIM ve LlamaIndex Entegrasyonu

Bu not defterinde, NVIDIA'nın yapay zeka modellerini ve LlamaIndex'i kullanarak güçlü bir Retrieval-Augmented Generation (RAG) hattı oluşturmayı göstereceğiz. NVIDIA'nın büyük dil modellerini ve gömme vektörlerini kullanacağız, bunları Azure AI Search ile vektör deposu olarak entegre edeceğiz ve arama kalitesini ve verimliliğini artırmak için RAG gerçekleştireceğiz.

## Faydalar
- **Ölçeklenebilirlik**: NVIDIA'nın büyük dil modellerini ve Azure AI Search'ü kullanarak ölçeklenebilir ve verimli bir geri çağırma sağlayın.
- **Maliyet Verimliliği**: Verimli vektör depolama ve hibrit arama teknikleriyle arama ve geri çağırmayı optimize edin.
- **Yüksek Performans**: Güçlü dil modellerini vektörleştirilmiş arama ile birleştirerek daha hızlı ve daha doğru yanıtlar alın.
- **Kalite**: İlgili belgelerle desteklenmiş dil modeli yanıtları sayesinde yüksek arama kalitesini koruyun.

## Ön Koşullar
- 🐍 Python 3.9 veya üzeri
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API Anahtarı, NVIDIA'nın büyük dil modellerine ve gömme vektörlerine NVIDIA NIM mikro hizmetleri aracılığıyla erişim için

## Kapsanan Özellikler
- ✅ NVIDIA Dil Modeli Entegrasyonu (kullanacağımız model: [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA Gömme Vektörleri (kullanacağımız model: [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Azure AI Search Gelişmiş Geri Çağırma Modları
- ✅ LlamaIndex ile Belge İndeksleme
- ✅ NVIDIA Dil Modelleri ile Azure AI Search ve LlamaIndex kullanarak RAG

Haydi başlayalım!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Kurulum ve Gereksinimler
Python sürümü >3.10 kullanarak bir Python ortamı oluşturun.

## Başlarken!


NVIDIA AI Foundation modellerini kullanmak için bir `NVIDIA_API_KEY` almanız gerekiyor:
1) [NVIDIA](https://build.nvidia.com/explore/discover) ile ücretsiz bir hesap oluşturun.
2) Tercih ettiğiniz modeli seçin.
3) Giriş bölümünde Python sekmesini seçin, ardından **Get API Key** butonuna tıklayın ve **Generate Key** butonuna basın.
4) Oluşturulan anahtarı kopyalayın ve NVIDIA_API_KEY olarak kaydedin. Bu noktadan itibaren uç noktalara erişiminiz olmalı.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## LLM ve Gömme Kullanarak RAG Örneği
### 1) LLM'i Başlat
`llama-index-llms-nvidia`, NVIDIA'nın LLM bağlayıcısı olarak bilinir ve NVIDIA API kataloğunda bulunan uyumlu modellerle bağlantı kurmanıza ve bu modellerden çıktı almanıza olanak tanır. Sohbet tamamlama modellerinin listesini burada bulabilirsiniz: https://build.nvidia.com/search?term=Text-to-Text

Burada **mixtral-8x7b-instruct-v0.1** modelini kullanacağız.


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Gömlemeyi Başlat

`llama-index-embeddings-nvidia`, NVIDIA'nın Embeddings bağlayıcısı olarak da bilinir, NVIDIA API kataloğunda bulunan uyumlu modellerle bağlantı kurmanıza ve bu modellerden üretim yapmanıza olanak tanır. Gömleme modeli olarak `nvidia/nv-embedqa-e5-v5` seçildi. Metin gömme modellerinin bir listesini buradan görebilirsiniz: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Azure AI Arama Vektör Deposu Oluşturun


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Verileriniz üzerinde sorular sormak için bir Sorgu Motoru oluşturun

Azure AI Search'te saf vektör arama kullanarak ve yanıtı LLM'mize (Phi-3.5-MOE) dayandırarak yapılan bir sorgu örneği:


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Azure AI Arama'da hibrit arama kullanan bir sorgu burada.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Vektör Arama Analizi
LLM yanıtı, Rusya'nın borsa piyasasıyla ilgili kaynak metinde belirtilen önemli ekonomik sonuçları doğru bir şekilde aktarmaktadır. Özellikle, Rus borsasının %40 değer kaybettiğini ve devam eden durum nedeniyle işlemlerin durdurulduğunu ifade etmektedir. Bu yanıt, kaynakta verilen bilgilerle iyi bir şekilde örtüşmekte olup, LLM'nin Rusya'nın eylemleri ve uygulanan yaptırımlar sonucunda borsa üzerindeki etkileri doğru bir şekilde belirlediğini ve özetlediğini göstermektedir.

#### Kaynak Düğümleri Yorumu
Kaynak düğümleri, uluslararası yaptırımlar nedeniyle Rusya'nın karşılaştığı ekonomik sonuçları ayrıntılı bir şekilde açıklamaktadır. Metin, Rus borsasının %40 değer kaybettiğini ve işlemlerin durdurulduğunu vurgulamaktadır. Ayrıca, Ruble'nin değer kaybı ve Rusya ekonomisinin daha geniş çapta izole edilmesi gibi diğer ekonomik etkilerden de bahsedilmektedir. LLM yanıtı, bu düğümlerden kritik noktaları etkili bir şekilde özetleyerek, sorguda talep edilen borsa etkisine odaklanmıştır.


Şimdi, Hibrit Arama'nın sağlam bir cevap vermediği bir sorguya bakalım:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Hibrit Arama: LLM Yanıt Analizi
Hibrit Arama örneğindeki LLM yanıtı, sağlanan bağlamın Rusya'nın Ukrayna'yı işgal ettiği kesin tarihi belirtmediğini gösteriyor. Bu yanıt, LLM'nin kaynak belgelerdeki mevcut bilgileri kullandığını ancak metinde kesin detayların eksikliğini kabul ettiğini ifade ediyor.

Yanıt, bağlamda Rusya'nın saldırganlığıyla ilgili olayların bahsedildiğini doğru bir şekilde belirtiyor, ancak spesifik işgal tarihini belirtmiyor. Bu durum, LLM'nin sağlanan bilgiyi anlama yeteneğini ve içerikteki boşlukları fark etme becerisini ortaya koyuyor. LLM, kullanıcıyı kesin tarih için harici kaynaklara veya tarihsel kayıtlara başvurmaya yönlendirerek, bilgilerin eksik olduğu durumlarda temkinli bir yaklaşım sergiliyor.

### Kaynak Düğümleri Analizi
Hibrit Arama örneğindeki kaynak düğümleri, ABD'nin Rusya'nın Ukrayna'daki eylemlerine verdiği yanıtı tartışan bir konuşmadan alıntılar içeriyor. Bu düğümler, işgalin daha geniş jeopolitik etkisini ve ABD ile müttefiklerinin işgale karşı aldığı adımları vurguluyor, ancak spesifik işgal tarihinden bahsetmiyor. Bu durum, bağlamda kesin tarih bilgisinin eksikliğini doğru bir şekilde belirleyen LLM yanıtıyla uyumlu.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hibrit ve Yeniden Sıralama: LLM Yanıt Analizi
Hibrit ve Yeniden Sıralama örneğinde, LLM yanıtı, olayın konuşmanın yapılmasından altı gün önce gerçekleştiğini belirterek ek bağlam sağlar. Bu, LLM'nin konuşmanın zamanlamasına dayanarak işgal tarihini çıkarabildiğini gösterir, ancak kesinlik için konuşmanın tam tarihini bilmesi gerektiğini de ortaya koyar.

Bu yanıt, bağlam ipuçlarını kullanarak daha bilgilendirici bir cevap verme yeteneğinin geliştiğini gösterir. Yeniden sıralamanın avantajını vurgular; burada LLM, daha alakalı bilgilere erişebilir ve öncelik verebilir, böylece istenen detaya (örneğin, işgal tarihi) daha yakın bir tahmin sunabilir.

### Kaynak Düğümleri Analizi
Bu örnekteki kaynak düğümleri, Rusya'nın işgalinin zamanlamasına referanslar içerir ve özellikle işgalin konuşmadan altı gün önce gerçekleştiğini belirtir. Tam tarih açıkça belirtilmemiş olsa da, düğümler LLM'nin daha incelikli bir yanıt vermesini sağlayan zamansal bağlam sunar. Bu detayın dahil edilmesi, yeniden sıralamanın LLM'nin sağlanan bağlamdan bilgi çıkarmasını ve yorumlamasını nasıl geliştirebileceğini gösterir, böylece daha doğru ve bilgilendirici bir yanıt elde edilir.


**Not:**  
Bu not defterinde, NVIDIA API Kataloğu'ndan NVIDIA NIM mikro hizmetlerini kullandık.  
Yukarıdaki API'ler, `NVIDIA (llms)`, `NVIDIAEmbedding` ve [Azure AI Search Semantic Hybrid Retrieval (yerleşik yeniden sıralama)](https://learn.microsoft.com/azure/search/semantic-search-overview).  
Unutmayın, yukarıdaki API'ler kendi barındırılan mikro hizmetleri de destekleyebilir.  

**Örnek:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")
```



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
